## This code is used only on websites written in JAVA Scrpit

### Step-1:- Create images Folder in same directory
### Step-2:- Download and Extract  and Copy in same directory Then  Run "Chromedriver.exe"


* https://chromedriver.chromium.org/
* Download chrome Driver

### For Firefox
* Download https://github.com/mozilla/geckodriver/releases

In [61]:
os.listdir()

['.ipynb_checkpoints',
 'chromedriver.exe',
 'Image Scrapping- Google Images.ipynb',
 'images']

In [5]:
# !pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [56]:
import os
import time
from selenium import webdriver
import requests

In [57]:
def fetch_image_urls(query:str,max_links_to_fetch:int,wd:webdriver,sleep_between_interactions:int=1 ):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)
        
    search_url="https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_I=img"
    
    wd.get(search_url.format(q=query))
    
    image_urls=set()
    image_count=0
    results_start=0
    
    while image_count<max_links_to_fetch:
        scroll_to_end(wd)
        
        #get all image thumbnail results
        thumbnail_results=wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results=len(thumbnail_results)
        
        print(f"Found:{number_results} search results. Extracting links from{results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the realimage behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue
            
            # extract image urls
            actual_images=wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
            
            image_count=len(image_urls)
            
            if len(image_urls)>=max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links Done...")
                break
        else:
            print("Found:",len(image_urls),"image links, looking for more..")
            time.sleep(30)
            return
            load_more_button=wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("documnet.querySelector('.mye4qd').click();")
            
        results_start=len(thumbnail_results)
    return image_urls
            
            

In [58]:
def persist_image(folder_path:str,url:str,counter):
    try:
        image_content=requests.get(url).content
    except Expection as e:
        print(f"Error-Could not download{url} - {e}")
        
    try:
        f=open(os.path.join(folder_path,'jpg' + '_'+str(counter)+ ".jpg"),'wb')
        f.write(image_content)
        f.close()
        print(f"Success - saved {url} - as{folder_path}")
    except Exception as e:
        print(f"Error - Could not save {url} - {w}")
        

In [59]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=10):
    target_folder=os.path.join(target_path,'_'.join(search_term.lower().split(' ')))
    if not os.path.exists(target_folder):
        os.makedirs(target_folder) # make dir if not present
        
    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term,number_images,wd=wd,sleep_between_interactions=0.5)
        
    counter=0
    for elem in res:
        persist_image(target_folder,elem,counter)
        counter+=1

In [62]:
# Inputs are given here

DRIVER_PATH='./chromedriver'
search_term='cat' 
# num of images you can pass it from here by default it is 10 if you are not passing
# number_images=10

search_and_download(search_term=search_term,driver_path=DRIVER_PATH)

Found:100 search results. Extracting links from0:100
Found: 10 image links Done...
Success - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTen03JQ7G1WboCraBA-bGx-eGf5IKxFmD55Q&usqp=CAU - as./images\cat
Success - saved https://d2c7ipcroan06u.cloudfront.net/wp-content/uploads/2020/04/Cat-representational-image-e1587622908176-696x391.jpg - as./images\cat
Success - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRU93-KmbhrU6DiSk82dfejzCjR9vNH8k7IDw&usqp=CAU - as./images\cat
Success - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRc8jp9Vwce_IE4m8bCH86nM7Vy9qYiwWZ0-g&usqp=CAU - as./images\cat
Success - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQBhSu2ijY76GLl_I1AFoVdlvfmLuW2Z7tPwg&usqp=CAU - as./images\cat
Success - saved https://static.toiimg.com/thumb/msid-72295960,imgsize-545889,width-800,height-600,resizemode-75/72295960.jpg - as./images\cat
Success - saved https://www.hindustantimes.com/rf/image_size_1200x900/HT/p2/2020/04